In [2]:
import pystan
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
sns.set(rc={'figure.figsize':(14, 10)})

## 16.3

In [4]:
df = pd.read_csv('data/stan_survival.csv', index_col=0)

### 16.3.2

$ p(Y) = \frac{\sqrt[3]{b}} {\Gamma\left(\frac{4}{3}\right)}exp(-bY^3)$

$ log(p) = log \left( \frac{\sqrt[3]{b}} {\Gamma\left(\frac{4}{3}\right) }exp(-bY^3)\right)$

$ log(p) = log \left( \frac{\sqrt[3]{b}} {\Gamma\left(\frac{4}{3}\right)} \right) -bY^3$

### 16.3.3 

In [7]:
ocode = """functions{
    real my_prob_lpdf(real Y, real b){
        return log(cbrt(b)/tgamma(4/3))-b*Y^3;
    }
}
"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4571326d135b2d69323dc47bcc281c0e NOW.


### 16.3.4

In [10]:
ocode = """
functions{
    real my_prob_lpdf(real Y, real b){
        return log(cbrt(b)/tgamma(4/3))-b*Y^3;
    }
}

data {
    int N;
    real Y[N];
}

parameters {
    real<lower=0> b;
}

model {
    for(i in 1:N)
        Y[i] ~ my_prob(b);
}
"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c6c57b65eca12585f05fe98c1a361537 NOW.


In [11]:
data = {'N': df.shape[0], 'Y': df.x}

In [13]:
fit = sm.sampling(data=data, iter=1000, chains=4, seed=1)
fit

Inference for Stan model: anon_model_c6c57b65eca12585f05fe98c1a361537.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
b      2.42  5.3e-3   0.14   2.15   2.33   2.42   2.51   2.69    668    1.0
lp__ -38.54    0.02   0.74 -40.72 -38.75 -38.23 -38.05  -38.0    920    1.0

Samples were drawn using NUTS at Mon Dec 21 01:15:23 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).